In [44]:
import pandas as pd
import censusdata
import geopandas as gpd
import json
from matplotlib import pyplot as plt

In [65]:
df = pd.read_csv('./2020-03-22-weekly-patterns.csv.gz', compression='gzip')

In [3]:
df.head(20)

,safegraph_place_id,location_name,street_address,city,region,postal_code,brands,naics_code,date_range_start,date_range_end,...,visits_by_each_hour,visitor_home_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_week_brand,device_type,iso_country_code
0,sg:0012c9c7f30b4369aa0a0230d961caed,Alternative Health Rehab Center,G3285 Beecher Rd,Flint,MI,48532,NaN,621330.0,2020-03-22T00:00:00-04:00,2020-03-29T00:00:00-04:00,...,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",{},"{""US"":6}",10148.0,19.0,"{""<5"":0,""5-20"":5,""21-60"":2,""61-240"":0,"">240"":0}","{""Benzer Pharmacy"":50,""Halo Burger"":33}","{""Dollar Tree"":26,""Halo Burger"":14,""Merrill Ly...","{""android"":5,""ios"":0}",US
1,sg:003436cf1d8845e991a7ce26f3661488,HP Car Accessories & ATV Sales,414 E Tyler St,Athens,TX,75751,NaN,441228.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"[0,0,0,0,0,0,0,0,0,0,1,0,0,1,2,0,0,0,0,0,0,0,0...",{},"{""US"":26}",11166.0,18.5,"{""<5"":1,""5-20"":25,""21-60"":17,""61-240"":5,"">240"":0}","{""Valero Energy"":33,""Shell Oil"":32,""Walmart"":2...","{""Walmart"":43,""Brookshire's Grocery Company"":2...","{""android"":16,""ios"":11}",US
2,sg:004c5a930cb645578e3cb928f9551323,Circle K,3709 E Houston Ave,Visalia,CA,93292,Circle K,445120.0,2020-03-22T00:00:00-07:00,2020-03-29T00:00:00-07:00,...,"[0,0,0,0,0,0,0,0,0,1,2,0,1,2,1,1,1,2,0,0,2,0,0...","{""061070013011"":9,""061070013023"":8,""0610700130...","{""US"":67}",4400.0,6.5,"{""<5"":9,""5-20"":76,""21-60"":7,""61-240"":4,"">240"":4}","{""WinCo Foods"":37,""Burger King"":24,""Costco Who...","{""Costco Wholesale Corp."":19,""Starbucks"":17,""F...","{""android"":45,""ios"":30}",US
3,sg:004d8fbf598141eda682027947cb11b1,Restaurant Ambra,705 S 4th St,Philadelphia,PA,19147,NaN,722511.0,2020-03-22T00:00:00-04:00,2020-03-29T00:00:00-04:00,...,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",{},{},NaN,14.0,"{""<5"":0,""5-20"":1,""21-60"":1,""61-240"":0,"">240"":0}",{},{},"{""android"":0,""ios"":0}",US
4,sg:005df5896ac249b2b08b89d0f04686ac,AAR Equine Trading,5206 Meadow Canyon Dr,Sugar Land,TX,77479,NaN,316992.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"[0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,2,0,2,0,0,0...",{},"{""US"":5}",47334.0,281.0,"{""<5"":0,""5-20"":2,""21-60"":5,""61-240"":9,"">240"":21}","{""The Home Depot"":19,""Walmart"":15}","{""99 Ranch Market"":9,""Freddy's Frozen Custard""...","{""android"":5,""ios"":6}",US
5,sg:005efd20816b44ca9ebad675077c7c91,CarX Auto Service,1223 Sagamore Pkwy W,West Lafayette,IN,47906,CarX Auto Service,811111.0,2020-03-22T00:00:00-04:00,2020-03-29T00:00:00-04:00,...,"[0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0...",{},"{""US"":6}",4165.0,5.0,"{""<5"":1,""5-20"":5,""21-60"":0,""61-240"":0,"">240"":0}",{},"{""Walmart"":17,""David's Bridal"":17,""J.D. Byride...","{""android"":0,""ios"":0}",US
6,sg:005fae987c6943f8bcabd7cc8762e655,CHAPMAN CHEVROLET L L C,1717 E Baseline Rd,Tempe,AZ,85283,Chevrolet,441110.0,2020-03-22T00:00:00-07:00,2020-03-29T00:00:00-07:00,...,"[0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,2,0,0,0,0,0...",{},"{""US"":31}",16417.0,80.0,"{""<5"":1,""5-20"":24,""21-60"":8,""61-240"":17,"">240""...","{""Bashas' Supermarkets"":20,""Sportsman's Wareho...","{""Fry's Food & Drug Stores"":47,""Circle K"":32,""...","{""android"":31,""ios"":5}",US
7,sg:005fb57f0eab45a0975f144c19c95dc2,Dallas Tortilla & Tamale Factory,213 Sh 342 Ste 405,Red Oak,TX,75154,NaN,722511.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",{},{},NaN,158.0,"{""<5"":0,""5-20"":2,""21-60"":0,""61-240"":1,"">240"":2}",{},{},"{""android"":0,""ios"":0}",US
8,sg:009a93737eae482782775064ea18f661,The Stubborn Mule,252 Main Ave,Clinton,IA,52732,NaN,722511.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0...","{""190450007002"":8}","{""US"":21}",1429.0,5.0,"{""<5"":5,""5-20"":19,""21-60"":0,""61-240"

In [9]:
df = df[df.city=='Chicago']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27347 entries, 238 to 3643702
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   safegraph_place_id         27347 non-null  object 
 1   location_name              27347 non-null  object 
 2   street_address             27347 non-null  object 
 3   city                       27347 non-null  object 
 4   region                     27347 non-null  object 
 5   postal_code                27347 non-null  int64  
 6   brands                     4617 non-null   object 
 7   naics_code                 27286 non-null  float64
 8   date_range_start           27347 non-null  object 
 9   date_range_end             27347 non-null  object 
 10  raw_visit_counts           27347 non-null  int64  
 11  raw_visitor_counts         27347 non-null  int64  
 12  visits_by_day              27347 non-null  object 
 13  visits_by_each_hour        27347 non-null 

In [41]:
#Make a new column for number of devices with home census blocks:
home_cbgs_num_devices = []
home_cbgs_num_cbgs = []
for i in range(len(df)):

    home_cbgs_dict = json.loads(df.iloc[i]['visitor_home_cbgs'])
    home_cbgs_num_cbgs.append(len(home_cbgs_dict))
    sum_devices = 0
    if home_cbgs_dict:
        for val in home_cbgs_dict.values():
            sum_devices += val
    home_cbgs_num_devices.append(sum_devices)
df['home_cbgs_num_devices'] = home_cbgs_num_devices
df['home_cbgs_num_cbgs'] = home_cbgs_num_cbgs
df.head(30)

,safegraph_place_id,location_name,street_address,city,region,postal_code,brands,naics_code,date_range_start,date_range_end,...,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_week_brand,device_type,iso_country_code,home_cbgs_num_devices,home_cbgs_num_cbgs
238,sg:0f73620eb5f74465978ad218d13e5213,Crossroads,1519 N Milwaukee Ave,Chicago,IL,60622,Crossroads,453310.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"{""US"":7}",5109.0,67.0,"{""<5"":1,""5-20"":3,""21-60"":2,""61-240"":5,"">240"":1}","{""BP"":33}","{""Target"":39,""Red Mango"":29,""Binny's Beverage ...","{""android"":0,""ios"":5}",US,0,0
406,sg:1968f4cb1b2b4c09a5640a84ef8561f0,Chicago Police Department,1718 S State St,Chicago,IL,60616,NaN,921190.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"{""US"":43}",15725.0,40.5,"{""<5"":1,""5-20"":32,""21-60"":44,""61-240"":30,"">240...","{""Mariano's"":12,""PNC Financial Services"":12,""S...","{""Mariano's"":31,""PNC Financial Services"":20,""S...","{""android"":32,""ios"":17}",US,0,0
533,sg:224472669e094453b56375b924998377,Municipal Park,Municipal Park,Chicago,IL,60473,NaN,712190.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"{""US"":10}",9389.0,36.0,"{""<5"":0,""5-20"":8,""21-60"":4,""61-240"":5,"">240"":3}","{""Freddy's Frozen Custard"":50,""Aurelio's Pizza...","{""Speedway"":19,""BP"":12,""Burger King"":12,""Aurel...","{""android"":10,""ios"":5}",US,0,0
601,sg:266785b403cf4c8aa012ace24c4551b2,Latin Patio Restaurant,3115 N Central Ave,Chicago,IL,60634,NaN,722511.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,{},NaN,146.0,"{""<5"":0,""5-20"":2,""21-60"":0,""61-240"":3,"">240"":2}",{},{},"{""android"":0,""ios"":0}",US,0,0
676,sg:2a30e79a029344a99393cc8bb74392e4,Tanuki,3006 N Sheffield Ave,Chicago,IL,60657,NaN,722511.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"{""US"":13}",12389.0,32.0,"{""<5"":1,""5-20"":9,""21-60"":3,""61-240"":6,"">240"":5}","{""Target"":98}","{""Subway"":16,""Mariano's"":12,""Jewel-Osco"":12,""B...","{""android"":10,""ios"":6}",US,0,0
685,sg:2aa0c4e2f761410fb1bb96d40e449ced,Playas Nayaritas,6000 W Grand Ave,Chicago,IL,60639,NaN,722511.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,{},NaN,20.5,"{""<5"":0,""5-20"":3,""21-60"":2,""61-240"":1,"">240"":0}","{""Dunkin'"":24}","{""Subway"":48,""Starbucks"":47,""LA Fitness"":33,""C...","{""android"":0,""ios"":0}",US,0,0
769,sg:2fc7cec6ff1f48cc8f6df5843e62f164,Northcroft Park,1365 South Ridge Road,Chicago,IL,60045,NaN,712190.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"{""US"":6}",21971.0,8.0,"{""<5"":0,""5-20"":5,""21-60"":0,""61-240"":1,"">240"":1}",{},"{""LA Fitness"":17,""YMCA"":17,""Orangetheory Fitne...","{""android"":0,""ios"":0}",US,0,0
945,sg:3b52a40083fd47f8a299ef020b2d97ab,La Petite Pastry,5610 W 63rd St,Chicago,IL,60638,NaN,311811.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"{""US"":8}",8068.0,15.5,"{""<5"":0,""5-20"":5,""21-60"":2,""61-240"":3,"">240"":0}","{""Menard's"":100}","{""ALDI"":32,""The Vitamin Shoppe"":22,""CITGO"":22,...","{""android"":7,""ios"":0}",US,0,0
959,sg:3c815a262cbe440dabfcaec0d861dcec,Foremost Liquor Lakeview,3014 N Ashland Ave,Chicago,IL,60657,NaN,445310.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"{""US"":12}",2133.0,9.0,"{""<5"":0,""5-20"":12,""21-60"":1,""61-240"":0,"">240"":0}",{},"{""Jewel-Osco"":31,""Target"":12,""Angelo Caputo's ...","{""android"":5,""ios"":8}",US,0,0
1024,sg:40734dece20e403b8e74dcdc893637f4,Fragola Yogurt And Ice Cream,3207 N Cental Ave,Chicago,IL,60634,NaN,722515.0,2020-03-22T00:00:00-05:00,2020-03-29T00:00:00-05:00,...,"{""US"":7}",1587.0,158.0,"{""<5"":0,""5-20"":9,""21-60"":1,""61-240"":5,"">240"":10}","{""Tony's Finer Foods"":100,""Dunkin'"":33,""Sam's ...","{""Cricket Wireless"":20,""BP"":19,""McDonald's"":13...","{""android"":5,""ios"":5}",US,0,0


In [58]:
df['home_cbgs_num_devices'].value_counts().head(20)

0     26332
5       322
6       160
7        83
8        53
10       45
9        37
11       31
12       29
13       26
15       17
16       14
14       10
20       10
18        9
17        8
21        8
19        7
26        6
22        6
Name: home_cbgs_num_devices, dtype: int64

In [4]:
# Note: IL FIPS = 17
acs_example = censusdata.download("acs5", 2015, censusdata.censusgeo(
    [("state", "17"), ("county", "031"), ("block group", "*")]), ["B02001_001E", "GEO_ID"])
acs_example["geo_12"] = acs_example["GEO_ID"].map(lambda x: str(x)[-12:])

In [5]:
acs_example.head()

,B02001_001E,GEO_ID,geo_12
"Block Group 1, Census Tract 202, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:020200> block group:1",1359,1500000US170310202001,170310202001
"Block Group 2, Census Tract 202, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:020200> block group:2",1459,1500000US170310202002,170310202002
"Block Group 3, Census Tract 202, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:020200> block group:3",1841,1500000US170310202003,170310202003
"Block Group 4, Census Tract 202, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:020200> block group:4",1975,1500000US170310202004,170310202004
"Block Group 1, Census Tract 203.01, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:020301> block group:1",1590,1500000US170310203011,170310203011


In [6]:
il_geo_12 = acs_example['geo_12'].unique()
df = df[df['origin_census_block_group'].isin(il_geo_12)]

KeyError: 'origin_census_block_group'

In [60]:
df2 = pd.read_csv('./cleaned_IL_sd.csv.gz', compression='gzip')

In [61]:
df2.head(20)

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home
0,170311201001,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,98,3343.0,"{""16001-50000"":5,""0"":52,"">50000"":9,""2001-8000""...","{""16001-50000"":211,"">50000"":680,""<1000"":20,""20...",50,1282,"{""721-1080"":15,""361-720"":7,""61-360"":2,""<60"":10...","[79,77,72,76,73,76,73,70,66,65,67,67,66,64,68,...",4,2,"{""170310815003"":1,""170359724003"":1,""1703112010...",1,0,158,"{""21-45"":4,""481-540"":4,""46-60"":1,""721-840"":1,""...",100,"{""26-50"":2,""51-75"":9,""0-25"":12,"">100"":2,""76-10..."
1,170311604003,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,41,4500.0,"{""16001-50000"":4,""0"":14,"">50000"":2,""2001-8000""...","{""16001-50000"":31,"">50000"":358,""<1000"":538,""20...",14,841,"{""721-1080"":9,""361-720"":1,""61-360"":3,""<60"":8,""...","[24,22,24,23,23,23,22,19,20,18,13,16,18,17,19,...",1,1,"{""170313301004"":1,""170312215001"":1,""1703180830...",1,11,85,"{""21-45"":1,""481-540"":1,""46-60"":1,""721-840"":2,""...",97,"{""26-50"":1,""51-75"":6,""0-25"":7,"">100"":2,""76-100..."
2,170312002002,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,64,3760.0,"{""16001-50000"":1,""0"":34,"">50000"":1,""2001-8000""...","{""16001-50000"":65,"">50000"":1072,""<1000"":134,""2...",32,999,"{""721-1080"":8,""361-720"":6,""61-360"":8,""<60"":12,...","[39,40,41,40,43,40,45,43,37,41,36,38,33,43,40,...",1,1,"{""170318114014"":1,""170318116002"":1,""1703120020...",1,0,109,"{""21-45"":6,""481-540"":4,""46-60"":2,""721-840"":1,""...",100,"{""26-50"":1,""51-75"":6,""0-25"":12,"">100"":3,""76-10..."
3,170314212001,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,28,8379.0,"{""16001-50000"":4,""0"":13,""2001-8000"":1,""1-1000""...","{""16001-50000"":25,""<1000"":143,""2001-8000"":79,""...",10,684,"{""721-1080"":2,""361-720"":1,""61-360"":6,""<60"":7,""...","[16,13,14,13,9,12,11,14,11,12,9,12,8,13,9,7,9,...",1,1,"{""170314402012"":1,""170312306004"":1,""1703124280...",1,11,54,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""<20"":1...",84,"{""0-25"":6,""76-100"":10,""51-75"":1,""26-50"":1}"
4,170314913001,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,34,7725.0,"{""16001-50000"":6,""0"":14,"">50000"":1,""2001-8000""...","{""16001-50000"":22,"">50000"":12,""<1000"":241,""200...",12,272,"{""721-1080"":3,""361-720"":4,""61-360"":3,""<60"":16,...","[17,19,16,15,18,13,16,15,10,12,11,11,10,11,14,...",1,1,"{""180890207001"":1,""170313802001"":1,""1703149030...",1,15,65,"{""21-45"":1,""481-540"":1,""1201-1320"":1,""301-360""...",94,"{""26-50"":1,""51-75"":3,""0-25"":9,"">100"":1,""76-100..."
5,170318241223,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,143,3976.0,"{""16001-50000"":13,""0"":69,"">50000"":3,""2001-8000...","{""16001-50000"":91,"">50000"":89,""<1000"":69,""2001...",68,1042,"{""721-1080"":21,""361-720"":12,""61-360"":20,""<60"":...","[105,108,107,101,99,101,90,92,88,90,83,82,79,8...",9,1,"{""170318407001"":1,""170318435001"":1,""1703182530...",3,6,228,"{""21-45"":7,""481-540"":1,""541-600"":1,""46-60"":1,""...",99,"{""26-50"":1,""51-75"":16,""0-25"":13,"">100"":5,""76-1..."
6,170318248002,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,96,5970.0,"{""16001-50000"":10,""0"":36,"">50000"":3,""2001-8000...","{""16001-50000"":127,"">50000"":15,""<1000"":263,""20...",34,882,"{""721-1080"":3,""361-720"":10,""61-360"":10,""<60"":2...","[63,64,64,65,64,64,57,54,55,45,47,46,50,56,48,...",4,1,"{""170318274003"":1,""170318247022"":2,""1704384260...",1,20,228,"{""21-45"":1,""481-540"":2,""541-600"":1,""46-60"":11

In [64]:
#Make a new column for number of devices with destination census blocks:
dest_cbgs_num_devices = []
dest_cbgs_num_cbgs = []
for i in range(len(df2)):

    dest_cbgs_dict = json.loads(df2.iloc[i]['destination_cbgs'])
    dest_cbgs_num_cbgs.append(len(dest_cbgs_dict))
    sum_devices = 0
    if dest_cbgs_dict:
        for val in dest_cbgs_dict.values():
            sum_devices += val
    dest_cbgs_num_devices.append(sum_devices)
df2['dest_cbgs_num_devices'] = dest_cbgs_num_devices
df2['dest_cbgs_num_cbgs'] = dest_cbgs_num_cbgs
df2.head(30)

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,...,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,dest_cbgs_num_devices,dest_cbgs_num_cbgs
0,170311201001,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,98,3343.0,"{""16001-50000"":5,""0"":52,"">50000"":9,""2001-8000""...","{""16001-50000"":211,"">50000"":680,""<1000"":20,""20...",50,1282,"{""721-1080"":15,""361-720"":7,""61-360"":2,""<60"":10...",...,2,"{""170310815003"":1,""170359724003"":1,""1703112010...",1,0,158,"{""21-45"":4,""481-540"":4,""46-60"":1,""721-840"":1,""...",100,"{""26-50"":2,""51-75"":9,""0-25"":12,"">100"":2,""76-10...",154,48
1,170311604003,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,41,4500.0,"{""16001-50000"":4,""0"":14,"">50000"":2,""2001-8000""...","{""16001-50000"":31,"">50000"":358,""<1000"":538,""20...",14,841,"{""721-1080"":9,""361-720"":1,""61-360"":3,""<60"":8,""...",...,1,"{""170313301004"":1,""170312215001"":1,""1703180830...",1,11,85,"{""21-45"":1,""481-540"":1,""46-60"":1,""721-840"":2,""...",97,"{""26-50"":1,""51-75"":6,""0-25"":7,"">100"":2,""76-100...",64,31
2,170312002002,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,64,3760.0,"{""16001-50000"":1,""0"":34,"">50000"":1,""2001-8000""...","{""16001-50000"":65,"">50000"":1072,""<1000"":134,""2...",32,999,"{""721-1080"":8,""361-720"":6,""61-360"":8,""<60"":12,...",...,1,"{""170318114014"":1,""170318116002"":1,""1703120020...",1,0,109,"{""21-45"":6,""481-540"":4,""46-60"":2,""721-840"":1,""...",100,"{""26-50"":1,""51-75"":6,""0-25"":12,"">100"":3,""76-10...",107,44
3,170314212001,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,28,8379.0,"{""16001-50000"":4,""0"":13,""2001-8000"":1,""1-1000""...","{""16001-50000"":25,""<1000"":143,""2001-8000"":79,""...",10,684,"{""721-1080"":2,""361-720"":1,""61-360"":6,""<60"":7,""...",...,1,"{""170314402012"":1,""170312306004"":1,""1703124280...",1,11,54,"{""481-540"":1,""1201-1320"":1,""301-360"":1,""<20"":1...",84,"{""0-25"":6,""76-100"":10,""51-75"":1,""26-50"":1}",45,25
4,170314913001,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,34,7725.0,"{""16001-50000"":6,""0"":14,"">50000"":1,""2001-8000""...","{""16001-50000"":22,"">50000"":12,""<1000"":241,""200...",12,272,"{""721-1080"":3,""361-720"":4,""61-360"":3,""<60"":16,...",...,1,"{""180890207001"":1,""170313802001"":1,""1703149030...",1,15,65,"{""21-45"":1,""481-540"":1,""1201-1320"":1,""301-360""...",94,"{""26-50"":1,""51-75"":3,""0-25"":9,"">100"":1,""76-100...",61,36
5,170318241223,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,143,3976.0,"{""16001-50000"":13,""0"":69,"">50000"":3,""2001-8000...","{""16001-50000"":91,"">50000"":89,""<1000"":69,""2001...",68,1042,"{""721-1080"":21,""361-720"":12,""61-360"":20,""<60"":...",...,1,"{""170318407001"":1,""170318435001"":1,""1703182530...",3,6,228,"{""21-45"":7,""481-540"":1,""541-600"":1,""46-60"":1,""...",99,"{""26-50"":1,""51-75"":16,""0-25"":13,"">100"":5,""76-1...",262,79
6,170318248002,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,96,5970.0,"{""16001-50000"":10,""0"":36,"">50000"":3,""2001-8000...","{""16001-50000"":127,"">50000"":15,""<1000"":263,""20...",34,882,"{""721-1080"":3,""361-720"":10,""61-360"":10,""<60"":2...",...,1,"{""170318274003"":1,""170318247022"":2,""1704384260...",1,20,228,"{""21-45"":1,""481-540"":2,""541-600"":1,""46-60"":11,...",96,"{""26-50"":12,""51-75"":10,""0-25"":14,"">100"":3,""76-...",190,79
7,170898528053,2020-04-03T00:00:00-05:00,2020-04-04T00:00:00-05:00,127,4848.0,"{""16001-50000"":8,""0"":57,"">50000"":9,""2001-8000""...","{""16001-50000"":109,"">50000"":115,""<1000"":131,""2...",56,842,"{""721-1080"":17,""361-720"":17,""61-